# 6156 Capstone Project

Group 5: Connor Lynch, Harrison Kim

In [1]:
# Clone directly from GitHub
!git clone https://github.com/harrisonkimdev/6156-capstone-project.git
%cd 6156-capstone-project

import os
print("Cloned Repository!")
print(f"Current location: {os.getcwd()}")

Cloning into '6156-capstone-project'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 80 (delta 1), reused 5 (delta 0), pack-reused 68 (from 2)
Receiving objects: 100% (80/80), 79.21 MiB | 42.71 MiB/s, done.
Resolving deltas: 100% (1/1), done.
remote: Total 80 (delta 1), reused 5 (delta 0), pack-reused 68 (from 2)
Receiving objects: 100% (80/80), 79.21 MiB | 42.71 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/Users/harrisonkim/code/repos/6156-capstone-project/notebooks/6156-capstone-project
Cloned Repository!
Current location: /Users/harrisonkim/code/repos/6156-capstone-project/notebooks/6156-capstone-project
/Users/harrisonkim/code/repos/6156-capstone-project/notebooks/6156-capstone-project
Cloned Repository!
Current location: /Users/harrisonkim/cod

/opt/miniconda3/envs/6156/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Install required packages and configure pose_ai module path

In [2]:
# 1. Install packages
%pip install -q --upgrade pip
%pip install -q numpy pandas scikit-learn xgboost opencv-python mediapipe

# 2. Configure path to import pose_ai module
import os
import sys
from pathlib import Path

# Add src folder to Python path
repo_root = Path.cwd()
src_path = repo_root / 'src'

if src_path.exists():
    if str(src_path) not in sys.path:
        sys.path.insert(0, str(src_path))
    os.environ.setdefault('PYTHONPATH', str(src_path) + (':' + os.environ.get('PYTHONPATH', '')))
    print(f"✓ src path added: {src_path}")
else:
    print(f"⚠️ Warning: src folder not found: {src_path}")
    print("   Please ensure the repository is cloned or src/ folder exists.")

# 3. Verify pose_ai import
try:
    import pose_ai as _pose_ai
    print(f'✓ pose_ai module imported successfully! (version: {getattr(_pose_ai, "__version__", "unknown")})')
except Exception as e:
    print(f'✗ pose_ai import failed: {e}')
    print('  Solutions:')
    print('  1. Verify the repository is cloned correctly')
    print('  2. Check if src/pose_ai/ folder exists')
    print('  3. Refer to "How to Use in Google Colab" cell above for repo setup')

# Display current working directory
print(f"\nCurrent working directory: {Path.cwd()}")
print(f"src added to Python path: {str(src_path) if src_path.exists() else 'N/A'}")


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
⚠️ Warning: src folder not found: /Users/harrisonkim/code/repos/6156-capstone-project/notebooks/6156-capstone-project/src
   Please ensure the repository is cloned or src/ folder exists.
✗ pose_ai import failed: No module named 'pose_ai'
  Solutions:
  1. Verify the repository is cloned correctly
  2. Check if src/pose_ai/ folder exists
  3. Refer to "How to Use in Google Colab" cell above for repo setup

Current working directory: /Users/harrisonkim/code/repos/6156-capstone-project/notebooks/6156-capstone-project
src added to Python path: N/A
Note: you may need to restart the kernel to use updated packages.
⚠️ Warning: src folder not found: /Users/harrisonkim/code/repos/6156-capstone-project/notebooks/6156-capstone-project/src
   Please ensure the repository is cloned or src/ folder exists.
✗ pose_ai import failed: No module named 'pose_ai'
  Solutions:
 

# Scripts Python Collection

This notebook aggregates the Python scripts from the `scripts` directory.

## scripts/extract_frames.py

In [ ]:
"""CLI for extracting frame sequences from climbing videos."""

from __future__ import annotations

import argparse
import logging
from pathlib import Path

from pose_ai.data import FrameExtractionResult, extract_frames_every_n_seconds, iter_video_files


LOGGER = logging.getLogger("pose_ai.scripts.extract_frames")


def configure_logging(verbose: bool) -> None:
    level = logging.DEBUG if verbose else logging.INFO
    logging.basicConfig(
        level=level,
        format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    )


def extract_from_directory(
    source_dir: Path,
    *,
    output_root: Path,
    interval_seconds: float,
    recursive: bool,
    overwrite: bool,
    write_manifest: bool,
) -> list[FrameExtractionResult]:
    results: list[FrameExtractionResult] = []
    for video_path in iter_video_files(source_dir, recursive=recursive):
        LOGGER.info("Processing %s", video_path)
        result = extract_frames_every_n_seconds(
            video_path,
            interval_seconds=interval_seconds,
            output_root=output_root,
            write_manifest=write_manifest,
            overwrite=overwrite,
        )
        LOGGER.info(
            "Saved %d frames for %s",
            result.saved_frames,
            video_path.name,
        )
        results.append(result)
    if not results:
        LOGGER.warning("No video files found in %s", source_dir)
    return results


def build_parser() -> argparse.ArgumentParser:
    parser = argparse.ArgumentParser(
        description="Extract frame sequences from climbing videos.",
    )
    parser.add_argument(
        "video_dir",
        type=Path,
        help="Directory containing source video files.",
    )
    parser.add_argument(
        "--output",
        type=Path,
        default=Path("data") / "frames",
        help="Directory where frame folders will be stored.",
    )
    parser.add_argument(
        "--interval",
        type=float,
        default=1.0,
        help="Seconds between captured frames (default: 1.0).",
    )
    parser.add_argument(
        "--recursive",
        action="store_true",
        help="Search for videos recursively.",
    )
    parser.add_argument(
        "--overwrite",
        action="store_true",
        help="Overwrite existing extracted frames.",
    )
    parser.add_argument(
        "--no-manifest",
        dest="write_manifest",
        action="store_false",
        help="Disable writing manifest.json files.",
    )
    parser.add_argument(
        "--verbose",
        action="store_true",
        help="Enable verbose logging.",
    )
    return parser


def main() -> None:
    parser = build_parser()
    args = parser.parse_args()

    configure_logging(verbose=args.verbose)
    extract_from_directory(
        args.video_dir,
        output_root=args.output,
        interval_seconds=args.interval,
        recursive=args.recursive,
        overwrite=args.overwrite,
        write_manifest=args.write_manifest,
    )


if __name__ == "__main__":
    main()


## scripts/run_feature_export.py

In [ ]:
"""CLI to export pose-derived features from manifests."""

from __future__ import annotations

import argparse
import json
from pathlib import Path

from pose_ai.service import export_features_for_manifest


def build_parser() -> argparse.ArgumentParser:
    parser = argparse.ArgumentParser(description="Export pose feature rows from pose_results.json.")
    parser.add_argument("manifest", type=Path, help="Path to manifest.json")
    parser.add_argument(
        "--holds",
        type=Path,
        help="Optional JSON describing holds (name -> coords, normalized, etc).",
    )
    parser.add_argument(
        "--out",
        type=Path,
        default=None,
        help="Output directory (defaults to manifest directory).",
    )
    return parser


def main() -> None:
    parser = build_parser()
    args = parser.parse_args()
    output_path = export_features_for_manifest(
        args.manifest,
        holds_path=args.holds,
        output_root=args.out,
    )
    print(f"Feature rows saved to {output_path}")


if __name__ == "__main__":
    main()


## scripts/run_pipeline.py

In [ ]:
"""End-to-end pipeline: extract frames, run pose estimation, features, segments, visualize."""

from __future__ import annotations

import argparse
import sys
from pathlib import Path

ROOT_DIR = Path(__file__).resolve().parents[1]
SRC_DIR = ROOT_DIR / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

from pose_ai.service import (  # type: ignore  # pylint: disable=wrong-import-position
    estimate_poses_from_manifest,
    export_features_for_manifest,
    generate_segment_report,
)
from extract_frames import extract_frames_every_n_seconds, iter_video_files  # type: ignore
from visualize_pose import visualize_pose_results  # type: ignore

# NOTE: For simplicity we call into script helpers directly for pose/feature export
# and reuse service APIs for intermediate steps.

def build_parser() -> argparse.ArgumentParser:
    parser = argparse.ArgumentParser(description="Run entire pose analysis pipeline")
    parser.add_argument("video_dir", type=Path, help="Directory containing videos (.mp4, etc.)")
    parser.add_argument("--out", type=Path, default=Path("data/frames"), help="Frame output directory")
    parser.add_argument("--interval", type=float, default=1.0, help="Extraction interval (seconds)")
    parser.add_argument("--skip-visuals", action="store_true", help="Skip visualization step")
    return parser


def extract_frames(video_dir: Path, out_dir: Path, interval: float) -> list[Path]:
    manifests = []
    out_dir.mkdir(parents=True, exist_ok=True)
    for video_file in iter_video_files(video_dir):
        result = extract_frames_every_n_seconds(video_file, output_root=out_dir, interval_seconds=interval)
        manifests.append(result.frame_directory / "manifest.json")
    return manifests


def main() -> None:
    parser = build_parser()
    args = parser.parse_args()
    manifests = extract_frames(args.video_dir, args.out, args.interval)
    for manifest in manifests:
        print(f"Processing manifest {manifest}")
        estimate_poses_from_manifest(manifest)
        export_features_for_manifest(manifest)
        generate_segment_report(manifest)
        if not args.skip_visuals:
            frame_dir = manifest.parent
            visualize_pose_results(frame_dir / "pose_results.json")
    print("Pipeline completed.")


if __name__ == "__main__":
    main()


## scripts/run_pose_estimation.py

In [ ]:
"""CLI to run pose estimation on extracted frame sequences."""

from __future__ import annotations

import argparse
import json
import sys
from pathlib import Path

from pose_ai.pose import PoseEstimator
from pose_ai.service import estimate_poses_for_directory, estimate_poses_from_manifest


def build_parser() -> argparse.ArgumentParser:
    parser = argparse.ArgumentParser(description="Run MediaPipe pose estimation on frame manifests.")
    group = parser.add_mutually_exclusive_group(required=True)
    group.add_argument("--manifest", type=Path, help="Path to a manifest.json file.")
    group.add_argument("--frames-root", type=Path, help="Directory containing extracted frame folders.")
    parser.add_argument(
        "--json",
        action="store_true",
        help="Print pose results as JSON instead of a textual summary.",
    )
    parser.add_argument(
        "--no-save",
        action="store_true",
        help="Disable writing pose_results.json files alongside frames.",
    )
    return parser


def frames_to_dict(frames):
    return [
        {
            "image_path": str(frame.image_path),
            "timestamp_seconds": frame.timestamp_seconds,
            "detection_score": frame.detection_score,
            "landmarks": [
                {
                    "name": landmark.name,
                    "x": landmark.x,
                    "y": landmark.y,
                    "z": landmark.z,
                    "visibility": landmark.visibility,
                }
                for landmark in frame.landmarks
            ],
        }
        for frame in frames
    ]


def main() -> None:
    parser = build_parser()
    args = parser.parse_args()

    estimator = PoseEstimator()
    try:
        if args.manifest:
            frames = estimate_poses_from_manifest(
                args.manifest,
                estimator=estimator,
                save_json=not args.no_save,
            )
            if args.json:
                print(json.dumps(frames_to_dict(frames), indent=2))
            else:
                print(f"Processed {len(frames)} frames from {args.manifest}")
        else:
            results = estimate_poses_for_directory(
                args.frames_root,
                estimator=estimator,
                save_json=not args.no_save,
            )
            if args.json:
                payload = {manifest: frames_to_dict(frames) for manifest, frames in results.items()}
                print(json.dumps(payload, indent=2))
            else:
                for manifest, frames in results.items():
                    print(f"{manifest}: {len(frames)} frames")
    except ModuleNotFoundError as exc:
        parser.error(
            f"{exc}. Ensure mediapipe is installed in your environment (e.g. `pip install mediapipe`)."
        )
    finally:
        estimator.close()


if __name__ == "__main__":
    main()


## scripts/run_segment_report.py

In [ ]:
"""CLI to generate segment-level metrics."""

from __future__ import annotations

import argparse

from pose_ai.service import generate_segment_report


def build_parser() -> argparse.ArgumentParser:
    parser = argparse.ArgumentParser(description="Export segment metrics (COM, joints, contacts)")
    parser.add_argument("manifest", type=str, help="Path to manifest.json")
    parser.add_argument("--holds", type=str, help="Optional holds JSON path", default=None)
    return parser


def main() -> None:
    parser = build_parser()
    args = parser.parse_args()
    metrics = generate_segment_report(args.manifest, holds_path=Path(args.holds) if args.holds else None)
    print(f"Saved {len(metrics)} segments")


if __name__ == "__main__":
    main()


## scripts/run_segmentation.py

In [ ]:
"""CLI to run rule-based segmentation over extracted frame manifests."""

from __future__ import annotations

import argparse
import json
from pathlib import Path

from pose_ai.service import segment_video_from_manifest, segment_videos_under_directory


def build_parser() -> argparse.ArgumentParser:
    parser = argparse.ArgumentParser(description="Produce rest/movement segments from frame manifests.")
    group = parser.add_mutually_exclusive_group(required=True)
    group.add_argument("--manifest", type=Path, help="Path to a manifest.json file.")
    group.add_argument("--frames-root", type=Path, help="Directory containing subfolders with manifest.json files.")
    parser.add_argument("--json", action="store_true", help="Print segmentation results as JSON.")
    return parser


def _segment_to_dict(segment):
    return {
        "start_time": segment.start_time,
        "end_time": segment.end_time,
        "label": segment.label,
        "duration": segment.duration,
        "frame_indices": segment.frame_indices,
    }


def main() -> None:
    parser = build_parser()
    args = parser.parse_args()

    if args.manifest:
        segments = segment_video_from_manifest(args.manifest)
        if args.json:
            print(json.dumps([_segment_to_dict(seg) for seg in segments], indent=2))
        else:
            print(f"Segments for {args.manifest}:")
            for seg in segments:
                print(
                    f"- {seg.label:9s} {seg.start_time:5.2f}s → {seg.end_time:5.2f}s "
                    f"(duration {seg.duration:4.2f}s, frames {seg.frame_indices})"
                )
    else:
        results = segment_videos_under_directory(args.frames_root)
        if args.json:
            payload = {
                manifest: [_segment_to_dict(seg) for seg in segments]
                for manifest, segments in results.items()
            }
            print(json.dumps(payload, indent=2))
        else:
            for manifest, segments in results.items():
                print(f"Segments for {manifest}:")
                for seg in segments:
                    print(
                        f"  - {seg.label:9s} {seg.start_time:5.2f}s → {seg.end_time:5.2f}s "
                        f"(duration {seg.duration:4.2f}s, frames {seg.frame_indices})"
                    )


if __name__ == "__main__":
    main()


## scripts/train_xgboost.py

In [ ]:
"""Train an XGBoost model on pose feature data (CLI)."""

from __future__ import annotations

import argparse
from pathlib import Path

from pose_ai.ml.xgb_trainer import TrainParams, params_from_dict, train_from_file


def _parse_args() -> argparse.Namespace:
    parser = argparse.ArgumentParser(description="Train XGBoost on pose feature rows.")
    parser.add_argument("features", type=Path, help="Path to pose_features.json")

    # Data/label
    parser.add_argument("--label-column", default="detection_score")
    parser.add_argument("--label-threshold", type=float, default=None)
    parser.add_argument("--drop-columns", nargs="*", default=["image_path"])
    parser.add_argument("--task", choices=["classification", "regression"], default="classification")
    parser.add_argument("--test-size", type=float, default=0.2)
    parser.add_argument("--random-state", type=int, default=42)

    # XGBoost hyperparameters
    parser.add_argument("--n-estimators", type=int, default=300)
    parser.add_argument("--learning-rate", type=float, default=0.05)
    parser.add_argument("--max-depth", type=int, default=4)
    parser.add_argument("--subsample", type=float, default=0.8)
    parser.add_argument("--colsample-bytree", type=float, default=0.8)
    parser.add_argument("--scale-pos-weight", type=float, default=None)
    parser.add_argument("--n-jobs", type=int, default=0)
    parser.add_argument("--tree-method", default=None, help="e.g., hist or gpu_hist")

    # Training behaviour
    parser.add_argument("--early-stopping-rounds", type=int, default=30)
    parser.add_argument("--eval-metric-cls", default="logloss")
    parser.add_argument("--eval-metric-reg", default="rmse")

    # Outputs
    parser.add_argument("--model-out", type=Path, default=Path("models/xgb_pose.json"))
    parser.add_argument("--metrics-out", type=Path, default=None)
    parser.add_argument("--feature-out", type=Path, default=None)
    parser.add_argument("--importance-out", type=Path, default=None)
    return parser.parse_args()


def main() -> None:
    args = _parse_args()
    params = params_from_dict(
        {
            "task": args.task,
            "label_column": args.label_column,
            "label_threshold": args.label_threshold,
            "drop_columns": args.drop_columns,
            "test_size": args.test_size,
            "random_state": args.random_state,
            "n_estimators": args.n_estimators,
            "learning_rate": args.learning_rate,
            "max_depth": args.max_depth,
            "subsample": args.subsample,
            "colsample_bytree": args.colsample_bytree,
            "scale_pos_weight": args.scale_pos_weight,
            "n_jobs": args.n_jobs,
            "tree_method": args.tree_method,
            "early_stopping_rounds": args.early_stopping_rounds,
            "eval_metric_cls": args.eval_metric_cls,
            "eval_metric_reg": args.eval_metric_reg,
            "model_out": args.model_out,
            "metrics_out": args.metrics_out,
            "feature_out": args.feature_out,
            "importance_out": args.importance_out,
        }
    )

    metrics = train_from_file(args.features, params)
    print(f"Model saved to {params.model_out}")
    print("Metrics:", metrics)


if __name__ == "__main__":
    main()


## scripts/visualize_pose.py

In [ ]:
"""Generate pose visualization overlays from pose_results.json."""

from __future__ import annotations

import argparse
import json
from pathlib import Path

import cv2

try:  # optional dependency for connection definitions
    from mediapipe.python.solutions.pose import PoseLandmark, POSE_CONNECTIONS
except ModuleNotFoundError:  # fallback if mediapipe not installed
    PoseLandmark = None
    POSE_CONNECTIONS = []


DEFAULT_CONNECTIONS = [
    ("left_shoulder", "right_shoulder"),
    ("left_shoulder", "left_elbow"),
    ("left_elbow", "left_wrist"),
    ("right_shoulder", "right_elbow"),
    ("right_elbow", "right_wrist"),
    ("left_hip", "right_hip"),
    ("left_shoulder", "left_hip"),
    ("right_shoulder", "right_hip"),
    ("left_hip", "left_knee"),
    ("left_knee", "left_ankle"),
    ("right_hip", "right_knee"),
    ("right_knee", "right_ankle"),
]


def build_parser() -> argparse.ArgumentParser:
    parser = argparse.ArgumentParser(description="Visualize pose landmarks on extracted frames.")
    parser.add_argument("pose_results", type=Path, help="Path to pose_results.json")
    parser.add_argument(
        "--output", type=Path, default=None,
        help="Directory to write visualized images (defaults to frame directory).",
    )
    parser.add_argument(
        "--include-missing", action="store_true",
        help="Include frames even when detection score/visibility is low.",
    )
    parser.add_argument(
        "--min-score",
        type=float,
        default=0.3,
        help="Minimum frame detection score required for visualization (default: 0.3).",
    )
    parser.add_argument(
        "--min-visibility",
        type=float,
        default=0.2,
        help="Minimum landmark visibility to draw a point/connection (default: 0.2).",
    )
    return parser


def _get_connections():
    if PoseLandmark is None or not POSE_CONNECTIONS:
        return DEFAULT_CONNECTIONS
    connections = []
    for a_idx, b_idx in POSE_CONNECTIONS:
        connections.append((PoseLandmark(a_idx).name.lower(), PoseLandmark(b_idx).name.lower()))
    return connections


def visualize_pose_results(
    pose_results_path: Path,
    output_dir: Path | None = None,
    *,
    include_missing: bool = False,
    min_score: float = 0.3,
    min_visibility: float = 0.2,
) -> int:
    payload = json.loads(pose_results_path.read_text(encoding="utf-8"))
    frames = payload.get("frames", [])
    count = 0
    connections = _get_connections()

    for frame in frames:
        image_path = Path(frame["image_path"])
        image = cv2.imread(str(image_path))
        if image is None:
            continue
        height, width = image.shape[:2]
        landmarks = frame.get("landmarks", [])
        detection_score = float(frame.get("detection_score", 0.0))
        if not include_missing and detection_score < min_score:
            continue
        if not landmarks and not include_missing:
            continue

        points = {}
        for landmark in landmarks:
            x = int(landmark["x"] * width)
            y = int(landmark["y"] * height)
            if landmark.get("visibility", 1.0) < min_visibility:
                continue
            points[landmark["name"]] = (x, y)
            cv2.circle(image, (x, y), 4, (0, 255, 0), -1)

        for start, end in connections:
            if start in points and end in points:
                cv2.line(image, points[start], points[end], (255, 0, 0), 2)

        target_dir = output_dir or image_path.parent / "visualized"
        target_dir.mkdir(parents=True, exist_ok=True)
        out_path = target_dir / f"{image_path.stem}_viz{image_path.suffix}"
        cv2.imwrite(str(out_path), image)
        count += 1
    return count


def main() -> None:
    parser = build_parser()
    args = parser.parse_args()
    processed = visualize_pose_results(
        args.pose_results,
        args.output,
        include_missing=args.include_missing,
        min_score=args.min_score,
        min_visibility=args.min_visibility,
    )
    print(f"Saved {processed} annotated frames")


if __name__ == "__main__":
    main()
